In [1]:
# Use SGD_TC to solve linear regression problem
import ObjectiveFunction as of
import helper_funcs as hf
import numpy as np
import torch
from torch.utils.data import Dataset
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm.notebook import tqdm
import itertools
import importlib
import copy
from ADAM_TC import ADAM_TC
import adaptive_algos as aa

seed = 1

In [2]:
class LinearRegressionDataset(Dataset):
    """
    Linear data with Gaussian noise. Parameterised by "seed".
    """
    def __init__(self, seed, dim, size, bounds, mtrue, btrue, sigma):

        np.random.seed(seed)
        torch.manual_seed(seed)

        sigma = torch.tensor(np.random.randn(dim)*sigma)
        print(sigma)

        self.Xdata = [torch.tensor(np.random.rand(dim)*bounds - 0.5*bounds) for _ in range(size)]
        self.Xdata = torch.vstack(self.Xdata)
        self.ydata = mtrue * self.Xdata + btrue
        self.ydata += sigma*torch.randn(size=(len(self.Xdata),1))
        self.data = (self.Xdata, self.ydata)
    
    def __len__(self):
        return len(self.Xdata)
    
    def __getitem__(self, idx):
        return self.Xdata[idx], self.ydata[idx]
    
    def shuffle_data(self, seed):
        torch.manual_seed(seed)
        shuffle_indices = torch.randperm(len(self.Xdata))
        self.Xdata = self.Xdata[shuffle_indices]
        self.ydata = self.ydata[shuffle_indices]
        self.data = (self.Xdata, self.ydata)

def cyclic_loader(dataloader):
    while True:
        for data in dataloader:
            yield data

In [3]:
databounds = 20
mtrue, btrue, sigma = 3.0, 4.0, 5.0
dataset = LinearRegressionDataset(seed=seed, dim=1, size=1000, bounds=databounds,
                                    mtrue=mtrue, btrue=btrue, sigma=sigma)

tensor([8.1217], dtype=torch.float64)


In [4]:
fig = px.scatter(x=dataset.Xdata.flatten(), y=dataset.ydata.flatten())
# fig.update_layout(
#     plot_bgcolor="#FFFFFF",
#     xaxis=dict(
#         title="Independent Variable (x)",
#         linecolor="#BCCCDC",
#     ),
#     yaxis=dict(
#         title="Dependent Variable (y)",
#         linecolor="#BCCCDC"
#     ),
#     font=dict(size=18)
# )

fig.update_layout(
    plot_bgcolor="#FFFFFF",
    xaxis=dict(
        title="Independent Variable (x)",
        linecolor="#BCCCDC",
    ),
    yaxis=dict(
        title="Dependent Variable (y)",
        linecolor="#BCCCDC"
    ),
    font=dict(size=18)
)
fig.show()

In [5]:
importlib.reload(of)
importlib.reload(aa)

batchsize = 32
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batchsize, shuffle=False)
train_cdl = cyclic_loader(dataloader)

max_iterations = 500
start = [5,-6]
lr = 0.01
opt_params = {'lr': lr}
def lr_annealer(opt, progress, lr):
        opt.param_groups[0]['lr'] = lr * (1 - progress)

metric = torch.nn.MSELoss()
model_sgd = of.LinearRegression(start=start, bounds=databounds) # bounds superfluous here
opt = torch.optim.SGD(params=model_sgd.parameters(), **opt_params)

losses_sgd, params = [], []
params.append(list(model_sgd.parameters())[0].detach().clone().numpy())
pbar = tqdm(range(max_iterations))

for i in pbar:
    model_sgd.train()
    Xdata_batch, ydata_batch = next(train_cdl)
    opt.zero_grad()
    ypred = model_sgd(Xdata_batch)
    loss = metric(ypred, ydata_batch)    
    loss.backward()
    opt.step()
    lr_annealer(opt, i/max_iterations, lr)

    losses_sgd.append(float(loss))
    params.append(list(model_sgd.parameters())[0].detach().clone().numpy())
    pbar.set_description(f'Epoch {i}: {loss}')

params_sgd = [list(_) for _ in list(zip(*params))]

  0%|          | 0/500 [00:00<?, ?it/s]

In [6]:
importlib.reload(aa)
importlib.reload(of)
filename = 'MD_LinearRegression_new.html'

# MD Params
lr = 0.05
height = 1.0    
width = 1
max_iterations = 2000
scale_annealer = lambda progress: (1 - progress)**2
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer}

start = [5,-6]
metric = torch.nn.MSELoss()

model = of.LinearRegression(start=start, bounds=databounds) # bounds superfluous here
opt = aa.Metadynamics(params=model.parameters(), func=model, **opt_params)

losses, params = [], []
params.append(list(model.parameters())[0].detach().clone().numpy())
pbar = tqdm(range(max_iterations))

for i in pbar:
    model.train()
    opt.zero_grad()
    Xdata = dataset.Xdata
    ydata = dataset.ydata
    ypred = model(Xdata)
    loss = metric(ypred, ydata)    
    loss.backward()
    opt.step()

    losses.append(float(loss))
    params.append(list(model.parameters())[0].detach().clone().numpy())
    pbar.set_description(f'Epoch {i}: {loss}')

params = [list(_) for _ in list(zip(*params))]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [6]:
importlib.reload(aa)
importlib.reload(of)

batchsize = 32
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batchsize, shuffle=False)
train_cdl = cyclic_loader(dataloader)

# SGD_TC Params
lr = 0.01
def lr_annealer(opt, progress, lr):
        opt.param_groups[0]['lr'] = lr * (1 - progress)
height = 1.0
width = 1
max_iterations = 500
scale_annealer = lambda progress: (1 - progress)
start = [5.0,-6.0]

opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer, 'adjust_dir': True}
metric = torch.nn.MSELoss()

model_levy = of.LinearRegression(start=start, bounds=databounds)
opt = aa.SGD_TC(params=model_levy.parameters(), func=model_levy, **opt_params)

losses_levy, params = [], []
params.append(list(model_levy.parameters())[0].detach().clone().numpy())

pbar = tqdm(range(max_iterations))
for i in pbar:
    model_levy.train()
    Xdata_batch, ydata_batch = next(train_cdl)
    opt.zero_grad()
    ypred = model_levy(Xdata_batch)
    loss = metric(ypred, ydata_batch)    
    loss.backward()
    opt.step()
    lr_annealer(opt, i/max_iterations, lr)

    losses_levy.append(float(loss))
    params.append(list(model_levy.parameters())[0].detach().clone().numpy())
    pbar.set_description(f'Epoch {i}: {loss}')

params_levy = [list(_) for _ in list(zip(*params))]

  0%|          | 0/500 [00:00<?, ?it/s]

In [7]:
def calc_contours(Z, divisions):
    m, M = np.min(Z), np.max(Z)
    size = (M - m)/divisions
    contours = dict(start=m, end=M, size=size)
    return contours

bounds = 10
xlim, ylim, res = (-bounds, bounds), (-bounds, bounds), 500

m = np.linspace(xlim[0], xlim[1], res)
b = np.linspace(ylim[0], ylim[1], res)
grid = [np.array(_) for _ in itertools.product(m,b)]
Z = [model_sgd.forward(dataset.Xdata, w=torch.tensor(_)) for _ in grid]
Z = [metric(ypred, dataset.ydata) for ypred in Z]
Z = np.array(Z).reshape(len(m), len(b)).T
gminimum_est = grid[np.argmin(Z)]

contours = calc_contours(Z,50)

In [27]:
# Plot SGD
fig1 = make_subplots(rows=1,cols=2)
fig1.add_trace(
    go.Scatter(x=np.arange(len(losses_sgd)), y=losses_sgd, line_color='green'), 
    row=1, col=1)

fig1.add_trace(
    go.Contour(x=m, y=b, z=Z, contours=contours),
    row=1, col=2,
)
fig1.add_trace(
    go.Scatter(x=params_sgd[0], 
               y=params_sgd[1],
               text=list(zip(losses_sgd, list(np.arange(0, len(losses_sgd))))),
               line_color='green',
               name='SGD'),
    row=1, col=2
)
fig1.update_layout(
    legend=dict(x=0.85,y=1, font=dict(size=20), title_text=''),
    xaxis=dict(tickfont=dict(size=20), titlefont=dict(size=20),
               title_text="Iteration Number", linecolor="#BCCCDC",),
    yaxis=dict(tickfont=dict(size=20), titlefont=dict(size=20), 
               title_text='MSE Loss', linecolor="#BCCCDC"),
    xaxis2=dict(tickfont=dict(size=20), titlefont=dict(size=20),
                title_text='Gradient (m)'),
    yaxis2=dict(tickfont=dict(size=20), titlefont=dict(size=20),
                title_text='Intercept (b)'),
    plot_bgcolor="#FFFFFF",
    font=dict(size=18)
)

title = 'ADAM/SGD' if not hasattr(opt, 'record') else str(opt.record)
title = title + str(list(model_sgd.parameters()))

fig1.update_layout(
    autosize=False,
    width=1600,
    height=800,
    title=title
)
X_data = dataset.Xdata.reshape(1,-1).squeeze()
y_true = dataset.ydata.reshape(1,-1).squeeze()
y_pred = model_sgd(dataset.Xdata).detach().reshape(1,-1).squeeze()

fig2 = hf.create_density_plot(params)
fig3 = go.Figure()
fig3.add_trace(
    go.Scatter(x=X_data, y=y_true, mode='markers', name='Data')
)
fig3.add_trace(
    go.Scatter(x=X_data, y=y_pred, line_color='green', name='SGD')
)

if hasattr(opt, 'alpha_record'):
    fig4 = px.scatter(opt.alpha_record)
    fig4.update_layout(
        yaxis_title='Alpha',
        xaxis_title='Iteration Number'
    )
    figs = [fig1, fig2, fig4, fig3]
else:
    figs = [fig1, fig2, fig3]


In [28]:
# Plot SGD_TC
fig1.add_trace(
    go.Scatter(x=np.arange(len(losses_levy)), y=losses_levy, line_color='red'), 
    row=1, col=1)

fig1.add_trace(
    go.Scatter(x=params_levy[0], 
               y=params_levy[1],
               text=list(zip(losses_levy, list(np.arange(0, len(losses_levy))))),
               line_color='red',
               name='Levy SGD'),
    row=1, col=2
)
fig1.add_trace(
    go.Scatter(x=[mtrue], y=[btrue], mode='markers',
               marker=dict(symbol='star', opacity=0.5, size=20, color='gold'),
               name='True Parameters'),
    row=1, col=2
)

title = 'ADAM/SGD' if not hasattr(opt, 'record') else str(opt.record)
title = title + str(list(model_levy.parameters()))

y_pred = model_levy(dataset.Xdata).detach().reshape(1,-1).squeeze()

fig2 = hf.create_density_plot(params)
fig3.add_trace(
    go.Scatter(x=X_data, y=y_pred, line_color='red', name='Levy SGD')
)
fig3.update_layout(
    legend=dict(x=0.92, y=0, font=dict(size=20), title_text=''),
    xaxis_title='x',
    yaxis_title='y',
    xaxis=dict(titlefont=dict(size=30), tickfont=dict(size=28),
               title="Independent Variable (x)", linecolor="#BCCCDC",),
    yaxis=dict(titlefont=dict(size=30), tickfont=dict(size=28),
               title="Dependent Variable (y)", linecolor="#BCCCDC"),
    plot_bgcolor="#FFFFFF",
    font=dict(size=30)
)

if hasattr(opt, 'alpha_record'):
    fig4 = px.scatter(opt.alpha_record)
    fig4.update_layout(
        yaxis_title='Alpha',
        xaxis_title='Iteration Number',
        xaxis=dict(titlefont=dict(size=20), tickfont=dict(size=20)),
        yaxis=dict(titlefont=dict(size=20), tickfont=dict(size=20))
    )
    figs = [fig1, fig2, fig4, fig3]
else:
    figs = [fig1, fig2, fig3]


In [29]:
filename = 'LinearRegression_final_results_new.html'
labels_to_show_in_legend = ["SGD", "Levy SGD", 'True Parameters']

for trace in fig1['data']: 
    if (not trace['name'] in labels_to_show_in_legend):
        trace['showlegend'] = False
hf.figures_to_html(figs, filename)

In [ ]:
"""
Main questions
- levy noise - power law vs levy-stable distribution
    - do we want levy flight or specifically this brand of levy noise
    https://link.springer.com/referenceworkentry/10.1007/978-0-387-30440-3_310
    https://www.pnas.org/doi/full/10.1073/pnas.2001548117
- what problems next: 
- non-convex problem next (e.g. quadratic polynomial fitting)
- apply to LSTM deep learning network
- compare with metadynamics + Parisi (tempering/simulated annealing)
- collect convergence statistics
"""

In [30]:
# Convergence Statistics (convex optimisation)
# Find when error first lower than some bound
importlib.reload(of)
importlib.reload(aa)

<module 'adaptive_algos' from 'c:\\Users\\karls\\Documents\\GitHub\\honours\\adaptive_algos.py'>

In [31]:
def train_model_linear_regression(dataset, experiment_num, batch_size, max_iterations, start, 
                                  opt_algo, opt_params, store=True, convergence_test=False):
    
    new_dataset = copy.deepcopy(dataset)
    LinearRegressionDataset.shuffle_data(new_dataset, experiment_num)
    dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batchsize, shuffle=True)
    
    train_cdl = cyclic_loader(dataloader)
    metric = torch.nn.MSELoss()
    model = of.LinearRegression(start=start, bounds=databounds) # bounds superfluous here
    opt = opt_algo(params=model.parameters(), **opt_params)
    def lr_annealer(opt, progress, lr):
        opt.param_groups[0]['lr'] = lr * (1 - progress)
    initial_loss = metric(model(new_dataset.Xdata), new_dataset.ydata)
    convergence_threshold = convergence_test * initial_loss

    if store and not convergence_test:
        losses, params = [], []
        losses.append(initial_loss)
        params.append(list(model.parameters())[0].detach().clone().numpy())
    
    if convergence_test:
        pbar = range(max_iterations)
    else:
        pbar = tqdm(range(max_iterations))

    for i in pbar:
        model.train()
        Xdata_batch, ydata_batch = next(train_cdl)
        opt.zero_grad()
        ypred = model(Xdata_batch)
        loss = metric(ypred, ydata_batch)
        loss.backward()
        opt.step()
        lr_annealer(opt, i/max_iterations, lr)

        if convergence_test:
            if loss < convergence_threshold:
                break
        else:
            pbar.set_description(f'Epoch {i}: {loss}')
            if store:
                losses.append(float(loss))
                params.append(list(model.parameters())[0].detach().clone().numpy())
    
    if convergence_test:
        ypred_full = model(new_dataset.Xdata)
        loss_full = metric(ypred_full, new_dataset.ydata)
        return i, loss_full

    if store:
        params = [list(_) for _ in list(zip(*params))]
        return losses, params, model, opt
    else:
        return model, opt

In [34]:
n_experiments = 10000
convergence_test = 0.3
max_iterations = 100
start = [5.0, -6.0]
batchsize = 32

In [35]:
# SGD Params
lr = 0.01
opt_params = {'lr': lr}

cutoff_iterations_sgd = []
full_losses_sgd = []
experiments = tqdm(range(n_experiments))

for experiment in experiments:
    i, full_loss = train_model_linear_regression(dataset, experiment, batchsize, max_iterations, start,
                                                torch.optim.SGD, opt_params=opt_params, 
                                                store=False, convergence_test=convergence_test)
    cutoff_iterations_sgd.append(i)
    full_losses_sgd.append(full_loss)
    experiments.set_description(f'Cutoff Iteration {i}')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [48]:
# SGD_TC Params
importlib.reload(aa)
lr = 0.01
height = 1.0
width = 1
scale_annealer = lambda progress: 1 - progress
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer, 'adjust_dir': True}

cutoff_iterations_levy = []
full_losses_levy = []
experiments = tqdm(range(n_experiments))
    
for experiment in experiments:
    i, full_loss = train_model_linear_regression(dataset, experiment, batchsize, max_iterations, start,
                                                aa.SGD_TC, opt_params=opt_params, 
                                                store=False, convergence_test=convergence_test)
    cutoff_iterations_levy.append(i)
    full_losses_levy.append(full_loss)
    experiments.set_description(f'Cutoff Iteration {i}')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [49]:
cutoff_iterations_sgd_np = np.array(cutoff_iterations_sgd)
cutoff_iterations_sgd_np = cutoff_iterations_sgd_np[cutoff_iterations_sgd_np < 30]
fig1 = px.histogram(cutoff_iterations_sgd_np, nbins=100, color_discrete_sequence=['seagreen'])
fig1.update_layout(
    title='SGD Mean Cutoff: {}, Median Cutoff: {}'.format(np.mean(cutoff_iterations_sgd_np), np.median(cutoff_iterations_sgd_np))
)


In [50]:
cutoff_iterations_levy_np = np.array(cutoff_iterations_levy)
# cutoff_iterations_levy_np = cutoff_iterations_levy_np[cutoff_iterations_levy_np < 30]
fig2 = px.histogram(cutoff_iterations_levy_np, nbins=100, color_discrete_sequence=['maroon'])
fig2.update_layout(
    title='Levy SGD Mean Cutoff: {}, Median Cutoff: {}'.format(np.mean(cutoff_iterations_levy_np), np.median(cutoff_iterations_levy_np))
)

# mean increases to 22.4 if we turn off the data-informed anisotropy, 
# due to a heavier right tail in the distribution of cutoff iterations

In [51]:
import pandas as pd
cutoffs = pd.DataFrame({'SGD': cutoff_iterations_sgd, 'Levy SGD': cutoff_iterations_levy})
print(cutoffs.mean(), cutoffs.median())
fig = px.histogram(cutoffs, barmode="overlay")
fig.update_layout(
    height=800,
    width=1600,
    legend=dict(x=0.90,y=1, font=dict(size=20), title_text=''),
    xaxis_title='Iteration Number to Reach 30% Initial Loss',
    yaxis_title='Count',
    yaxis=dict(tickfont=dict(size=20), titlefont=dict(size=20)),
    xaxis=dict(tickfont=dict(size=20), titlefont=dict(size=20))
)
fig.show()


SGD         16.7177
Levy SGD    16.6874
dtype: float64 SGD         17.0
Levy SGD    16.0
dtype: float64


In [52]:
sum(full_losses_sgd)/len(full_losses_sgd), sum(full_losses_levy)/len(full_losses_levy)

(tensor(125.2613, dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(124.9109, dtype=torch.float64, grad_fn=<DivBackward0>))